In [37]:
!pip install bert
!pip install bert-for-tf2

In [38]:
import os
import math
import datetime
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm

In [39]:
from sklearn.metrics import confusion_matrix, classification_report
import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
# from bert.tokenization.bert_tokenization import AutoTokenizer

In [30]:
random_seed = 42
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

In [40]:
import csv
import json

# Function to convert CSV to JSON
def csv_to_json(csv_file, json_file):
    data = []
    with open(csv_file, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            data.append(row)

    with open(json_file, 'w') as json_file:
        json.dump(data, json_file, indent=4)

# Replace these file paths with your own
csv_file_path = 'Intent.csv'
json_file_path = 'data_full.json'

csv_to_json(csv_file_path, json_file_path)


In [41]:
df = pd.read_csv('Intent.csv')
# Drop rows with NaN values
df.dropna(inplace=True)

# Now, df_sheet1 contains no rows with NaN values
# Create a new DataFrame without NaN rows
df_sheet1 = df.dropna()
# Convert the DataFrame to a dictionary
data_dict=df_sheet1.to_dict(orient='list')
for key, values in data_dict.items():
    data_dict[key] = [value.lower() for value in values]

intents = data_dict
# Step 1: Data Collection and Preparation

# Define your data as a dictionary where keys are intents and values are lists of text examples.
custom_data = intents

# Step 1b: Data Preprocessing (Within the loop)
# Data preprocessing within the loop includes converting text to lowercase and removing special characters.

preprocessed_custom_data = {}  # To store preprocessed data

for intent, texts in custom_data.items():
    preprocessed_texts = []  # To store preprocessed texts for each intent
    for text in texts:
        # Data preprocessing steps:
        # 1. Convert text to lowercase
        text = text.lower()
        # 2. Remove special characters (keeping only letters and spaces)
        text = ''.join(e for e in text if e.isalnum() or e.isspace())
        preprocessed_texts.append(text)
    preprocessed_custom_data[intent] = preprocessed_texts

# # Example usage
# for intent, texts in preprocessed_custom_data.items():
#     for text in texts:
#         # Print the intent and preprocessed text
#         print(f"Intent: {intent}, Text: {text}")

## Data

In [45]:
import csv
import random

# Function to split data dictionary into train, test, and validate sets
def split_dict(data_dict, train_csv, test_csv, validate_csv, train_ratio, test_ratio):
    data = list(data_dict.values())
    header = list(data_dict.keys())

    # Shuffle the data for randomness
    random.shuffle(data)

    # Calculate split sizes
    total_rows = len(data)
    train_size = int(total_rows * train_ratio)
    test_size = int(total_rows * test_ratio)
    validate_size = total_rows - train_size - test_size

    # Split the data
    train_data = data[:train_size]
    test_data = data[train_size:train_size + test_size]
    validate_data = data[train_size + test_size:]

    # Write the split data to separate CSV files
    with open(train_csv, 'w', newline='') as train_file:
        csv_writer = csv.writer(train_file)
        csv_writer.writerow(header)
        csv_writer.writerows(zip(*train_data))

    with open(test_csv, 'w', newline='') as test_file:
        csv_writer = csv.writer(test_file)
        csv_writer.writerow(header)
        csv_writer.writerows(zip(*test_data))

    with open(validate_csv, 'w', newline='') as validate_file:
        csv_writer = csv.writer(validate_file)
        csv_writer.writerow(header)
        csv_writer.writerows(zip(*validate_data))

# Define data dictionary and split ratios
data_dict = preprocessed_custom_data
train_csv = 'train_data.csv'
test_csv = 'test_data.csv'
validate_csv = 'validate_data.csv'
train_ratio = 0.7  # 70% for training
test_ratio = 0.15  # 15% for testing

# Perform the split
split_dict(data_dict, train_csv, test_csv, validate_csv, train_ratio, test_ratio)


In [46]:
import pandas as pd

In [49]:
# df.to_csv('train_data.csv', index=False,header=('text','intent'))
train=pd.read_csv('train_data.csv')
train["greet"]

0    how to set an existing photo as a profile phot...
1        how to change photo as profile photo in media
2             want to convert a photo to profile photo
3    is there a way to set an existing photo to pro...
4                   photo to profile photo in my media
5      want to put a photo from media to profile photo
6                               photo to profile photo
7                  how can i set prfle photo frm media
Name: greet, dtype: object

In [52]:
# df.to_csv('val_data.csv', index=False,header=('text','intent'))
valid=pd.read_csv('validate_data.csv')
valid = valid.fillna('')
print(len(valid))
valid.head()

8


,goodbye,greet,date_of_birth_not_displaying_onet,onet_is_veryslow,profile_photo_not_visible,can_i_reset_myonet,name_not_displaying,new_postnot_createdoruploaded,postnowoption _is_not_working,edit_my_profile_name,...,all_my_media_onet,set_profile_media_onet,set_cover_media_onet,download_photo_media_onet,delete_photo_media_onet,create_schedule_post_onet,search_schedule_post,share_post,share_posttoclub,share_copylink
0,i m facing problem with o net can i reset my o...,how to search a schedule post,in my onet profile my birthdate is also hidden...,how can i see my blog in onet,how can i keep my profile privacy,how to upload a post,where can find my followers,how can i report channel in podcast,name not displaying it displays public,how can i exit from club,...,,,,,,,,,,
1,i need to reset my onet because im having issu...,searching schedule posts,my birthdate is also not visible in my onet pr...,what steps should i take to view my blog on on...,from where i can keep my profile private,i am unable to upload a post,where i can see who follows me,i want a report channel in podcast,no name is displayed it displays public,can i leave the club,...,,,,,,,,,,
2,my onet isnt working right so i need to reset it,a schedule post can be searched using this method,additionally my onet profile does not display ...,is there a way for me to access my blog throug...,where can i find my privacy settings,i upload a post not,can i see my followers,report channel in podcast,no ones name is mentioned just public,the best way to leave the club,...,,,,,,,,,,
3,my onet is giving me problems so i need to res...,what is the best way to search a schedule post,my birthdate is also hidden in my onet profile...,can you guide me on how to check my blog using...,what are my privacy settings,upload post,want to see where my followers are,report podcast channel in onet,just public is mentioned not anyones name,as a admin user how can i exist from club,...,,,,,,,,,,
4,what is the process to reset onet account,using the schedule post search,why my birthday is not displaying in onet profile,is it possible for me to see my blog on onet i...,i want to change my privacy settings where do ...,how to add a post,followers for me,what is the process for reporting a channel on...,name not showing it displays public,whr can i exist frm club,...,,,,,,,,,,


In [53]:
# df.to_csv('test_data.csv', index=False,header=('text','intent'))
test=pd.read_csv('test_data.csv')
test = test.fillna('')
print(len(test))
test.head()

8


,goodbye,greet,date_of_birth_not_displaying_onet,onet_is_veryslow,profile_photo_not_visible,can_i_reset_myonet,name_not_displaying,new_postnot_createdoruploaded,postnowoption _is_not_working,edit_my_profile_name,...,all_my_media_onet,set_profile_media_onet,set_cover_media_onet,download_photo_media_onet,delete_photo_media_onet,create_schedule_post_onet,search_schedule_post,share_post,share_posttoclub,share_copylink
0,what are the unique features have onet,how can i exit a club as admin,how do i change or modify my names wrongly spe...,how to create clubs,can i save podcast in onet,when i am trying to start posting uploading im...,how can i see all my photos and videos off on...,how do i share a post with the club,compare to other social networks onet in very ...,how do i view my entire list of friends on onet,...,,,,,,,,,,
1,how is the onet different from your competitors,process to exit the club as admin,how can i correct or change my names that were...,what are the steps to creating clubs,is there a way for me to save podcast in onet,after uploading images in onet post now option...,how do i view a list of all of my onet photos ...,is there a way to share a post with the club,onet is a very slow social network when compar...,i want to search my all friends in onet,...,,,,,,,,,,
2,how does onet differ from other application,where may a club exist admin,how can i fix names that were spelt incorrectl...,processes are involved in starting a club,would it be possible to save podcast onet,i am unable to post my picture as post now opt...,how can i view a list of every photo and video...,how can i share a post with the club,onet is significantly slower than other social...,how do i access my entire onet friend list,...,,,,,,,,,,
3,can you tell me what makes onet unique,i dont want to continue my club but i am a admin,how can i correct the misspelt names in my one...,to create a club where can i do that how,how to save podcast onet,post now option is not working in onet,where can i see all my photos,what is the best way to share a post with the ...,if compared to other social networks onet is a...,how do i find all friends on onet,...,,,,,,,,,,
4,in what ways is onet unique,how can one leave the club as an admin,how can i change the names in my onet profile ...,what are the procedures for starting clubs,is it possible to preserve a podcast on onet,because the post now option is not functioning...,how can i see all my uploaded videos,if i want to share a post with the club how do...,onet is very slow in comparison to other socia...,i want to view all my friends,...,,,,,,,,,,


In [54]:
train = train.append(valid).reset_index(drop=True)

<ipython-input-54-96d7b38ab442>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append(valid).reset_index(drop=True)


75

In [55]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=def05c9cfb699877f817aa05e12fc2c7c8e15a0612cabdc39bce1505711c2d61
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [56]:
# Install wget (if not already installed)


# Define the download URL for the BERT model and vocabulary
bert_model_url = "https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip"
vocab_url = "https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12/vocab.txt"

# Define the directory where you want to store the model
model_dir = "/content/model"

# Create the directory if it doesn't exist
import os
os.makedirs(model_dir, exist_ok=True)

# Download and unzip the BERT model
import wget
import zipfile

bert_zip_file = os.path.join(model_dir, "bert_model.zip")
wget.download(bert_model_url, bert_zip_file)

with zipfile.ZipFile(bert_zip_file, "r") as zip_ref:
    zip_ref.extractall(model_dir)

# Download the vocabulary file
vocab_file = os.path.join(model_dir, "uncased_L-12_H-768_A-12/vocab.txt")
wget.download(vocab_url, vocab_file)

# Clean up the zip file if needed
os.remove(bert_zip_file)


In [57]:
os.makedirs("model", exist_ok=True)
bert_model_name="uncased_L-12_H-768_A-12"
bert_ckpt_dir = os.path.join("model/", bert_model_name)
bert_ckpt_file = os.path.join(bert_ckpt_dir, "bert_model.ckpt")
bert_config_file = os.path.join(bert_ckpt_dir, "bert_config.json")

## Input Text Preparation

In [58]:
!pip install  --upgrade bert

In [59]:
!pip uninstall transformers
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.1 MB/s eta 0:00:00


In [60]:
preprocessed_custom_data
labels = list(preprocessed_custom_data.keys())
texts = list(preprocessed_custom_data.values())

In [70]:
from sklearn.model_selection import train_test_split
# Assuming you have your data and labels (X and y)
# X contains your features, and y contains your labels/targets

# Step 1: Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(labels, texts, test_size=0.2, random_state=42)

# Step 2: Further split your training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Now, you have split your data into three sets:
# X_train, y_train: Training data and labels (typically 60-80% of the data)
# X_val, y_val: Validation data and labels (typically 10-20% of the data)
# X_test, y_test: Testing data and labels (typically 10-20% of the data)

# The 'random_state' parameter ensures reproducibility; you can choose any integer.


In [72]:
len(X_train)

48

In [69]:
text_lists= texts
def prepare_data(self, labels, text_lists):
      x, y = [], []

      for label, texts in tqdm(zip(labels, text_lists)):
          label_id = self.classes.index(label)

          for text in texts:
              tokens = self.tokenizer.tokenize(text)
              tokens = ["[CLS]"] + tokens + ["[SEP]"]
              token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
              self.max_seq_len = max(self.max_seq_len, len(token_ids))
              x.append(token_ids)
              y.append(label_id)

      return np.array(x), np.array(y)

In [73]:
import numpy as np
from tqdm import tqdm
from typing import List
from transformers import AutoTokenizer

class DataPreparation:
  def __init__(self, texts, labels, tokenizer: AutoTokenizer, classes, max_seq_len=192):
      self.tokenizer = tokenizer
      self.max_seq_len = 0
      self.classes = classes

      (self.x, self.y) = self.prepare_data(texts, labels)  # Call prepare_data with texts and labels

      print("max seq_len", self.max_seq_len)
      self.max_seq_len = min(self.max_seq_len, max_seq_len)
      self.x = self.data_padding(self.x)

  # Modify prepare_data to take texts and labels
  def prepare_data(self, texts, labels):
      x, y = [], []
      for text, label in zip(texts, labels):
          tokens = self.tokenizer.tokenize(text)
          if len(tokens) > self.max_seq_len:
              tokens = tokens[:self.max_seq_len]
          input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
          x.append(input_ids)
          y.append(self.classes.index(label))
          if len(input_ids) > self.max_seq_len:
              self.max_seq_len = len(input_ids)
      return np.array(x), np.array(y)

  def data_padding(self, ids):
        x = []
        for input_ids in ids:
            input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
            input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
            x.append(np.array(input_ids))
        return np.array(x)



In [74]:
from transformers import AutoTokenizer

model_name = "bert-base-uncased"  # Model name for uncased BERT variants
tokenizer = AutoTokenizer.from_pretrained(model_name)


# model_name = '/content/model/uncased_L-12_H-768_A-12/'
# tokenizer = AutoTokenizer.from_pretrained(model_name)

## Model

In [75]:
def model_defination(max_seq_len, bert_ckpt_file):

    with tf.io.gfile.GFile(bert_config_file, "r") as reader:
        bc = StockBertConfig.from_json_string(reader.read())
        bert_params = map_stock_config_to_params(bc)
        bert_params.adapter_size = None
        bert = BertModelLayer.from_params(bert_params, name="bert")

    input_ids = keras.layers.Input(shape=(max_seq_len, ), dtype='int32', name="input_ids")
    bert_output = bert(input_ids)

    print("bert shape", bert_output.shape)

    cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
    cls_out = keras.layers.Dropout(0.5)(cls_out)
    logits = keras.layers.Dense(units=768, activation="tanh")(cls_out)
    logits = keras.layers.Dropout(0.5)(logits)
    logits = keras.layers.Dense(units=len(classes), activation="softmax")(logits)

    model = keras.Model(inputs=input_ids, outputs=logits)
    model.build(input_shape=(None, max_seq_len))

    load_stock_weights(bert, bert_ckpt_file)

    return model

In [76]:
classes = list(train.keys())

data,x,y = DataPreparation(X_train,y_train, tokenizer, classes)

ValueError: ignored

In [ ]:
data.train_x.shape

(2400, 29)

In [ ]:
data.train_x[0]

array([  101,  2054,  2051,  2003,  2009,  1999, 27377,  2175, 13639,
        1010,  3516,   102,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0])

In [ ]:
data.train_y[0]

0

In [ ]:
model = model_defination(data.max_seq_len, bert_ckpt_file)

bert shape (None, 29, 768)
Done loading 196 BERT weights from: model/uncased_L-12_H-768_A-12\bert_model.ckpt into <bert.model.BertModelLayer object at 0x00000187DE11DEE0> (prefix:bert). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 29)]              0         
_________________________________________________________________
bert (BertModelLayer)        (None, 29, 768)           108890112 
_________________________________________________________________
lambda (Lambda)              (None, 768)               0         
_________________________________________________________________
dropout (Dropout)            (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 768)               590592    
_________________________________________________________________
dropout_1 (Dropout)          (None, 768)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                15380 

In [ ]:
model.compile(
  optimizer=keras.optimizers.Adam(1e-5),
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")]
)

In [ ]:
log_dir = "log/intent_detection/" + datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[-3]
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir)

history = model.fit(
  x=data.train_x,
  y=data.train_y,
  validation_split=0.1,
  batch_size=16,
  shuffle=True,
  epochs=5,
  callbacks=[tensorboard_callback]
)

Epoch 1/5
135/135 [==============================] - 330s 2s/step - loss: 3.1890 - acc: 0.0660 - val_loss: 1.7071 - val_acc: 0.7583
Epoch 2/5
135/135 [==============================] - 313s 2s/step - loss: 1.5232 - acc: 0.5881 - val_loss: 0.2066 - val_acc: 0.9375
Epoch 3/5
135/135 [==============================] - 314s 2s/step - loss: 0.3383 - acc: 0.9356 - val_loss: 0.1594 - val_acc: 0.9500
Epoch 4/5
135/135 [==============================] - 314s 2s/step - loss: 0.1425 - acc: 0.9811 - val_loss: 0.1135 - val_acc: 0.9542
Epoch 5/5
135/135 [==============================] - 317s 2s/step - loss: 0.0884 - acc: 0.9890 - val_loss: 0.1136 - val_acc: 0.9583


In [ ]:
_, train_acc = model.evaluate(data.train_x, data.train_y)
_, test_acc = model.evaluate(data.test_x, data.test_y)

19/19 [==============================] - 27s 1s/step - loss: 0.0952 - acc: 0.9700


## Accuracy

In [ ]:
print("train acc", train_acc)
print("test acc", test_acc)

train acc 0.9929166436195374
test acc 0.9700000286102295


## Let's Try

In [ ]:
sentences = [
  "is it six o clock yet",
  "find me a hotel with good reviews in phoenix"
]
pred_tokens = map(tokenizer.tokenize, sentences)
pred_tokens = map(lambda tok: ["[CLS]"] + tok + ["[SEP]"], pred_tokens)
pred_token_ids = list(map(tokenizer.convert_tokens_to_ids, pred_tokens))
pred_token_ids = map(
  lambda tids: tids +[0]*(data.max_seq_len-len(tids)),
  pred_token_ids
)
pred_token_ids = np.array(list(pred_token_ids))
predictions = model.predict(pred_token_ids).argmax(axis=-1)
for text, label in zip(sentences, predictions):
    print("text:", text, "\nintent:", classes[label])
    print()

text: is it six o clock yet 
intent: time

text: find me a hotel with good reviews in phoenix 
intent: book_hotel

